In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from sklearn import cross_validation as cv
from neupy.algorithms import GRNN as grnn
from neupy.estimators import mse
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/evan/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dta = pd.read_excel('data_SARIMA.xlsx',index_col='Date')
dta['Jan'] = dta['Month']==1
dta.head()

,Death,Death_rate,Incidence,Incidence_rate,forecast,Year,Month,error_ARIMA,Jan
Date,,,,,,,,,
2004-01-01,67,0.005154,99466,7.651937,NaN,2004,1,NaN,True
2004-02-01,68,0.005231,84156,6.474136,NaN,2004,2,NaN,False
2004-03-01,111,0.008539,94360,7.259132,NaN,2004,3,NaN,False
2004-04-01,121,0.009309,91944,7.073268,NaN,2004,4,NaN,False
2004-05-01,151,0.011616,90379,6.952873,NaN,2004,5,NaN,False


In [3]:
data_train = dta.loc['2005-02-1':'2014-12-01'][['Incidence_rate','forecast','Jan']]
x_train = data_train.loc[:'2012-12-01'][['forecast','Jan']]
y_train = data_train.loc[:'2012-12-01']['Incidence_rate']
x_test = data_train.loc['2013-01-01':'2013-12-01'][['forecast','Jan']]
y_test = data_train['2013-01-01':'2013-12-01']['Incidence_rate']

In [4]:
def try_std(x):
    nn = grnn(std =x ,verbose=False)
    nn.train(x_train,y_train)
    y_pred = nn.predict(x_test)
    print(mse(y_pred,y_test))

In [5]:
for x in np.linspace(0.05,0.11,11):
        print(x)
        try_std(x)
        print('--\n')

0.05
0.08964853589096201
--

0.056
0.08298670764087872
--

0.062
0.0781118726822838
--

0.068
0.07466999925532512
--

0.074
0.07232624780464286
--

0.08
0.07081278940756351
--

0.086
0.06991727055392875
--

0.092
0.06947268425631777
--

0.098
0.0693501996129461
--

0.104
0.06945246941279726
--

0.11
0.06970721496022857
--



In [8]:
mod_GRNN = grnn(std = 0.1092 ,verbose=False)
mod_GRNN.train(x_train,y_train)
data_train['NN']=mod_GRNN.predict(data_train[['forecast','Jan']])
data_train.head()

,Incidence_rate,forecast,Jan,NN
Date,,,,
2005-02-01,6.701542,7.607165,False,7.403787
2005-03-01,10.011233,7.736467,False,7.998863
2005-04-01,10.302798,9.171980,False,9.715133
2005-05-01,9.333632,9.495360,False,9.281360
2005-06-01,9.282781,9.035324,False,8.781631


In [7]:
data_train.to_excel('data_GRNN.xlsx')